## Setup

In [1]:
import findspark
findspark.init()

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /opt/hive3/lib/hive-hcatalog-core-3.1.2.jar pyspark-shell'

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [4]:
from pyspark.sql.session import SparkSession

spark = SparkSession.builder\
            .appName("Indivividual Assignment")\
            .config("hive.metastore.uris","thrift://localhost:9083")\
            .config("spark.sql.warehouse.dir","hdfs://localhost:9000/user/hive/warehouse")\
            .enableHiveSupport()\
            .getOrCreate()

You should see your tables in the ouput of the following cell

In [5]:
spark.sql("use geonames")
spark.sql("show tables").toPandas()

,database,tableName,isTemporary
0,geonames,geonames_india_postalcodes_raw,False
1,geonames,geonames_india_postalcodes_std,False
2,geonames,geonames_india_raw,False
3,geonames,geonames_india_std,False


Just replace the statement and execute
(No need to include ; at the end)

In [6]:
spark.sql(
"""
select * 
from geonames_india_raw 
limit 5
"""
).toPandas()

,geonameid,name,ascii_name,alternate_names,latitude,longitude,feature_class,feature_code,country_code,country_code2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
0,1114940,Rāvi River,Ravi River,"Ravi,Ravi River,Rāvi,Rāvi River",30.62123,71.82683,H,STM,IN,PK,00,,,,0,,133,Asia/Kolkata,2017-06-06
1,1114942,Punjab Plains,Punjab Plains,Punjab Plains,30,75,T,PLN,IN,,00,,,,0,,206,Asia/Kolkata,2012-01-16
2,1114957,Jhelum River,Jhelum River,"Jhelum,Jhelum River,River Hydaspes,Veth River,jhelama,jہlm,جہلم,झेलम",31.16853,72.15066,H,STM,IN,PK,00,,,,0,,147,Asia/Kolkata,2020-11-11
3,1114958,Hindustan,Hindustan,"Hindustan,Hindustán",28,76,L,RGN,IN,"IN,PK",00,,,,0,,344,Asia/Kolkata,2006-06-23
4,1114965,Basantar River,Basantar River,"Basantar,Basantar Nala,Basantar Nāla,Basantar River",32.47452,75.01449,H,STM,IN,,00,,,,0,,300,Asia/Kolkata,2016-03-09


# Query - 8

In [7]:
spark.sql(
"""
select count(*), feature_class
from geonames_india_std
group by feature_class;
"""
).toPandas()

,count(1),feature_class
0,35680,T
1,38130,L
2,3328,V
3,2,U
4,2026,A
5,39772,S
6,958,R
7,1107904,P
8,69982,H


# Query - 9

In [8]:
spark.sql(
"""
select distinct(name),population from geonames_india_std order by population desc limit 10;
"""
).toPandas()

,name,population
0,6269134,1700000000
1,1269750,1352617328
2,1253626,199812341
3,1264418,112374333
4,1275715,104099452
5,1252881,91276115
6,1264542,72626809
7,1255053,72147030
8,1258899,68548437
9,1267701,61095297


# Query - 10

In [9]:
spark.sql(
"""
select name, feature_code from geonames_india_std where feature_code = 'AIRP';
"""
).toPandas()

,name,feature_code
0,1271477,AIRP
1,6301029,AIRP
2,6301030,AIRP
3,6301031,AIRP
4,6301032,AIRP
...,...,...
357,11703501,AIRP
358,11776669,AIRP
359,11928000,AIRP
360,12110496,AIRP


# Query - 11

In [10]:
spark.sql(
"""
select distinct(ascii_name), elevation from geonames_india_std where feature_code = 'MT' order by elevation desc limit 10;
"""
).toPandas()

,ascii_name,elevation
0,Nanda Devi,7816
1,Kamet,7756
2,Saser Kangri,7672
3,Kabru,7412
4,Badrinath,7138
5,P'ao-han-li Shan,7128
6,Trisul,7120
7,Purvi Dunagiri,7066
8,Dunagiri,7066
9,Kanggardo Rize,7060


# Query - 12

In [11]:
spark.sql(
"""
select name from geonames_india_std where (latitude = '19.0760' AND longitude = '72.8777') AND (feature_code = 'HSP');
"""
).toPandas()

,name


# Query - 13

In [12]:
spark.sql(
"""
select count(name), timezone from geonames_india_std group by timezone;
"""
).toPandas()

,count(name),timezone
0,1297776,Asia/Kolkata
1,6,


# Query - 14

In [13]:
spark.sql(
"""
select count(distinct postal_code), admin_name1 from geonames_india_postalcodes_std group by admin_name1 
order by count(distinct postal_code) desc;
"""
).toPandas()

,count(DISTINCT postal_code),admin_name1
0,2035,Tamil Nadu
1,1644,Uttar Pradesh
2,1583,Maharashtra
3,1418,Kerala
4,1343,Karnataka
5,1214,Andhra Pradesh
6,1147,West Bengal
7,1026,Gujarat
8,992,Rajasthan
9,922,Odisha


# Query - 15

In [14]:
spark.sql(
"""
select admin_name3, postal_code, count(feature_code) from geonames.geonames_india_std ind inner join 
geonames.geonames_india_postalcodes_std ind_post on ind.admin2_code = ind_post.admin_code2 where feature_code in 
('HSP','PRK','MTRO','SCH')
group by admin_name3, postal_code order by count(feature_code) desc limit 10;
"""
).toPandas()

,admin_name3,postal_code,count(feature_code)
0,Bhor,412206,592
1,Viramgam,382150,468
2,Dehradun,248001,460
3,Kannad,431103,440
4,Vaijapur,423701,420
5,Chakrata,248123,380
6,Junnar,410502,368
7,Baramati,413102,336
8,New Delhi,110018,336
9,New Delhi,110043,336
